# Numerical simulation for double-bracket quantum algorithms for quantum imaginary-time evolution (DB-QITE) using density matrix exponentiation (DME)

## 1. Initialization

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import sys
sys.path.append('..')
from functions.qite_gci import *
from functions.qite_dme import *
from functions.plot_bloch_sphere import *

# Plot config
plt.rcParams.update({
    'axes.labelsize': 16,
    'axes.titlesize': 16,
    'font.size': 16,
    'savefig.dpi': 75,
    'lines.linewidth': 2.0,
    'lines.markersize': 8,
    'legend.fontsize': 14,
    'legend.labelspacing': .3,
    'legend.columnspacing': .3,
    'legend.handletextpad': .1,
    'text.usetex': True,
    'font.family': "serif",
    'font.serif': "cm",
    'figure.figsize': [5, 3.5],
    'figure.autolayout': True
})

[Qibo 0.2.12|ERROR|2025-06-16 13:05:41]: The 'qibojit' backends' provider is not available. Check that a Python package named 'qibojit' is installed, and it is exposing valid Qibo backends.
[Qibo 0.2.12|ERROR|2025-06-16 13:05:41]: The 'qibojit' backends' provider is not available. Check that a Python package named 'qibojit' is installed, and it is exposing valid Qibo backends.
[Qibo 0.2.12|INFO|2025-06-16 13:05:41]: Using numpy backend on /CPU:0


---
## 2. Run DB-QITE with DME

### 2.1 Find optimal s
Import the optimal s array generated in `find_optimal_s_dbqite.ipynb`

In [2]:
optimal_s_Mk = np.load('optimal_s_Mk.npy')

### 2.2 k = 1

This section shows **DB-QITE** results for different initial states, parameterized by recursion steps \(k\) and DME Trotter steps \(M\).

- The **x-axis** shows the fidelity of each initial state with the ground state. These initial states are prepared by applying \( RX(\theta) \) rotations to \(\ket{0}\), varying only the **latitude** on the Bloch sphere. This is because energy—relevant for state reset—depends on **latitude**, not longitude.
- The **y-axis** shows the fidelity with the ground state **after DB-QITE**. Higher values (closer to 1) indicate better reset performance.

#### Parameters

- `Ms = [1, 2, 50]`: List of \(M\) values representing DME memory (Trotter) steps.
- `k = 1`: Number of DB-QITE recursion iterations.

#### 1. Generate Initial States

- `thetas` is a linear space of 100 values from 0 to 1.
- `sigma_x` is the Pauli-X matrix.
- `psis` is a list of quantum states generated by evolving the state \(|1\rangle\) under the Pauli-X Hamiltonian:
  
  \[
  \ket{\psi(\theta)} = e^{-i \frac{\pi}{2} \theta \sigma_x} \ket{1}
  \]

#### 2. Run DB-QITE for Each M

- For each \(M \in \{1, 2, 50\}\), the DB-QITE energy is computed using the optimal step size \(s\) stored in `optimal_s_Mk`.
- Final fidelity is computed as:

  \[
  F = \frac{1 - E}{2}
  \]

  where \(E\) is the final energy returned by the DB-QITE routine (closer to 1 implies worse fidelity, so subtracting and scaling brings it back to fidelity-like form).


In [3]:
s_list = np.linspace(0.01,1,100)
k = 1

# Initial state sweep
thetas = np.linspace(0, 1, 100)
sigma_x = np.array([[0, 1], [1, 0]])
psis = [expm(-1j * np.pi/2 * sigma_x * t) @ np.array([[0], [1]]) for t in thetas]
psis = np.asarray(psis)

# Baseline GCI energy results
res_init = [GCI(initial_psi, psi, 1, np.linspace(0.01, 1, 2)).execute_gci()[0][-1] for psi in psis]

# Run DB-QITE for each M with optimal s
energies = []
M=1
for s in s_list:
    res_DBQITE = [GCI(QITE_dme_select_M(M), psi, None, [s]).execute_gci()[0][-1] for psi in psis]
    energies.append(res_DBQITE)

fidelities = [[np.real((1 - E) / 2) for E in E_list] for E_list in energies]

0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of interval reached
0 0 0.01 --- Warning edge of inter

In [4]:
target_fidelity = 0.9

# Define reference state |0⟩⟨0|
state_0 = np.array([1, 0], dtype=complex)

# Fidelity with respect to |0⟩ for both pure and mixed states
def fidelity_with_0(state):
    return np.real(np.conj(state_0).T @ (state @ state.T.conj() if state.ndim == 2 and state.shape[1] == 1 else state) @ state_0)

# Initial state fidelities and RGCI outputs
fidelities_original = [fidelity_with_0(x) for x in psis]

In [5]:
im = plt.imshow(np.asarray(fidelities).T, aspect='auto', extent=[min(fidelities_original), max(fidelities_original), min(s_list), max(s_list)], origin='lower')
cbar = plt.colorbar(im)

# Add text to top and bottom
cbar.ax.text(1.5, 1.05, r'$|0\rangle$', ha='center', va='bottom', transform=cbar.ax.transAxes)
cbar.ax.text(1.5, -0.05, r'$|1\rangle$', ha='center', va='top', transform=cbar.ax.transAxes)

plt.axhline(0.79, linewidth=0.8)
plt.ylabel('Step size s')
plt.xlabel('Initial fidelity\nwith ground state')
plt.title('Final fidelities with ground state\nM=1,k=1')
plt.tight_layout()
plt.savefig('plots/M1k1_s_sweep.pdf')

### 2.2 k=2

In [6]:
# Run DB-QITE for each M with optimal s
energies = []
M=2
for s in s_list:
    res_DBQITE = [GCI(QITE_dme_select_M(M), psi, None, [s]).execute_gci()[0][-1] for psi in psis]
    energies.append(res_DBQITE)

fidelities = [[np.real((1 - E) / 2) for E in E_list] for E_list in energies]

In [7]:
target_fidelity = 0.9

# Define reference state |0⟩⟨0|
state_0 = np.array([1, 0], dtype=complex)

# Fidelity with respect to |0⟩ for both pure and mixed states
def fidelity_with_0(state):
    return np.real(np.conj(state_0).T @ (state @ state.T.conj() if state.ndim == 2 and state.shape[1] == 1 else state) @ state_0)

# Initial state fidelities and RGCI outputs
fidelities_original = [fidelity_with_0(x) for x in psis]

In [8]:
im = plt.imshow(np.asarray(fidelities).T, aspect='auto', extent=[min(fidelities_original), max(fidelities_original), min(s_list), max(s_list)], origin='lower')
cbar = plt.colorbar(im)

# Add text to top and bottom
cbar.ax.text(1.5, 1.05, r'$|0\rangle$', ha='center', va='bottom', transform=cbar.ax.transAxes)
cbar.ax.text(1.5, -0.05, r'$|1\rangle$', ha='center', va='top', transform=cbar.ax.transAxes)

plt.axhline(0.88, linewidth=0.8)
plt.ylabel('Step size s')
plt.xlabel('Initial fidelity\nwith ground state')
plt.title('Final fidelities with ground state\nM=2,k=1')
plt.tight_layout()
plt.savefig('plots/M2k1_s_sweep.pdf')